# Basic Model: Naive Bayes

### Import necessary packages

In [1]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

import mlflow
import logging 
import config 

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

from basic_functions import (
    get_preprocess_data,
    get_lemmatized_data,
    get_metrics
)

/Users/maren/Library/CloudStorage/OneDrive-Persönlich/Dokumente/neue_fische/Capstone Project/Capstone_project/backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup

In [2]:
MODEL_NAME = "Naive_Bayes_oversampling" 
TRACKING_URI = open("../.mlflow_uri").read().strip()
EXPERIMENT_NAME = config.EXPERIMENT_NAME

logging.basicConfig(format="%(asctime)s: %(message)s") # Configure logging format to show timestamp before every message

logger = logging.getLogger()
logger.setLevel(logging.INFO) # Only show logs that are INFO or more important (e.g., WARNING, ERROR) — but ignore DEBUG.

In [3]:
DATA_PATH = "../data/data_dropped_duplicates_small.csv"

### Get and process data

In [4]:
df = get_preprocess_data(DATA_PATH)

### Lemmatize text

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/maren/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
df = get_lemmatized_data(df)

### Train-test split

In [7]:
y = df["logical_fallacies"]
X = df["text"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y)

### Naive Bayes

In [8]:
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()
mlflow.set_tag("model_name", MODEL_NAME)
mlflow.set_tag("mlflow.runName", "naive bayes oversampling")
# mlflow.log_params(params)

In [9]:
#Pipeline for TF-IFD and Naive Bayes
pipeline_bayes = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('ros', RandomOverSampler(random_state=10)),
    ('nb', MultinomialNB()),
])

# Train the model
pipeline_bayes.fit(X_train, y_train)

# Get predictions on the resampled training data
X_res, y_res = pipeline_bayes.named_steps['ros'].fit_resample(
    pipeline_bayes.named_steps['tfidf'].transform(X_train), 
    y_train
)
y_res_pred = pipeline_bayes.named_steps['nb'].predict(X_res)

# Predict on test data
y_test_pred_bayes = pipeline_bayes.predict(X_test)

### Evaluation


In [10]:
def log_metrics(cr, split):
    for key, value in cr.items():
        if (key == "accuracy"):
                # print(f"{split}_{key}", round(value,2))
                mlflow.log_metric(f"{split}_{key}", value)
        else:
            for metric in value:
                mlflow.log_metric(f"{split}_{key}_{metric}", value.get(metric))
                # print(f"{split}_{key}_{metric}", round(value.get(metri

In [11]:
logger.info('get_train_metrics')
classification_report_train = get_metrics(y_res, y_res_pred)
log_metrics(classification_report_train, "train")

INFO:root:get_train_metrics
INFO:basic_functions:classification_report
INFO:basic_functions:confusion_matrix


                       precision    recall  f1-score   support

           ad_hominem       0.93      0.91      0.92      3800
  appeal_to_authority       0.84      0.98      0.90      3800
    appeal_to_emotion       0.96      0.82      0.88      3800
        false_dilemma       0.88      0.97      0.92      3800
faulty_generalization       0.89      0.90      0.90      3800
                 none       0.87      0.77      0.82      3800

             accuracy                           0.89     22800
            macro avg       0.90      0.89      0.89     22800
         weighted avg       0.90      0.89      0.89     22800

[[3473  118   41   35   51   82]
 [  12 3711   13   10   10   44]
 [ 107  226 3113  117  128  109]
 [  15    5    5 3691   25   59]
 [  24   78   14  104 3436  144]
 [  88  289   73  232  192 2926]]


In [12]:
logger.info('get_test_metrics')
classification_report_test = get_metrics(y_test, y_test_pred_bayes)
log_metrics(classification_report_test, "test")

INFO:root:get_test_metrics
INFO:basic_functions:classification_report
INFO:basic_functions:confusion_matrix


                       precision    recall  f1-score   support

           ad_hominem       0.53      0.48      0.50       333
  appeal_to_authority       0.26      0.68      0.38       219
    appeal_to_emotion       0.67      0.38      0.49       504
        false_dilemma       0.42      0.72      0.53       290
faulty_generalization       0.43      0.50      0.46       420
                 none       0.80      0.62      0.70      1629

             accuracy                           0.57      3395
            macro avg       0.52      0.56      0.51      3395
         weighted avg       0.64      0.57      0.59      3395

[[ 160   41   27   23   40   42]
 [   8  149   10    9   10   33]
 [  42   95  193   47   72   55]
 [   2   13    2  208   16   49]
 [  27   51   21   43  209   69]
 [  64  218   34  161  136 1016]]


In [13]:
mlflow.end_run()

🏃 View run naive bayes oversampling at: http://127.0.0.1:5001/#/experiments/823412171152425451/runs/45ea536892f14b35977beac85102d9fe
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/823412171152425451
